# GIQL operators demo

This notebook demonstrates the 7 core GIQL operators:
- **Binary predicates**: INTERSECTS, WITHIN, CONTAINS
- **Aggregation operators**: MERGE, CLUSTER
- **Distance operators**: DISTANCE, NEAREST

For each operator, we:
1. Write a GIQL query
2. Transpile it to standard SQL
3. Execute the SQL with DuckDB

In [1]:
import duckdb
import sqlglot
import sqlparse

from giql import transpile

## Load BED files

Load two ENCODE ChIP-seq peak files into DuckDB.

In [2]:
conn = duckdb.connect()

# Load BED files with standard column names
bed_columns = """columns={
    'chrom': 'VARCHAR', 'start': 'INTEGER', 'end': 'INTEGER',
    'name': 'VARCHAR', 'score': 'INTEGER', 'strand': 'VARCHAR',
    'signal_value': 'DOUBLE', 'p_value': 'DOUBLE',
    'q_value': 'DOUBLE', 'peak': 'INTEGER'
}"""

for table_name, file_path in [
    ("features_a", ".ENCFF199YFA.bed"),
    ("features_b", ".ENCFF205OKL.bed"),
]:
    conn.execute(f"""
        CREATE TABLE {table_name} AS
        SELECT * FROM read_csv('{file_path}', delim='\t', header=false, {bed_columns})
    """)

print("Features A:", conn.execute("SELECT COUNT(*) FROM features_a").fetchone()[0], "intervals")
print("Features B:", conn.execute("SELECT COUNT(*) FROM features_b").fetchone()[0], "intervals")
conn.execute("SELECT * FROM features_a LIMIT 5").fetchdf()

Features A: 60893 intervals
Features B: 46196 intervals


,chrom,start,end,name,score,strand,signal_value,p_value,q_value,peak
0,chr17,27381032,27381306,.,1000,.,461.10586,-1.0,5.08726,136
1,chr7,25565932,25566199,.,1000,.,457.60096,-1.0,5.08726,133
2,chr7,143215117,143215391,.,1000,.,443.34712,-1.0,5.08726,139
3,chr2,164600030,164600306,.,1000,.,442.64446,-1.0,5.08726,137
4,chr15,56246046,56246359,.,1000,.,438.56423,-1.0,5.08726,154


---

## 1. INTERSECTS operator

Find features in A that **overlap** with features in B.

**GIQL Query**: `SELECT a.* FROM features_a a, features_b b WHERE a.interval INTERSECTS b.interval`

In [3]:
# Define GIQL query
giql_query = """
    SELECT DISTINCT a.*
    FROM features_a a, features_b b
    WHERE a.interval INTERSECTS b.interval
"""

# Transpile to SQL
sql = transpile(giql_query)
print("Transpiled SQL:")
print(
    sqlparse.format(
        sql, reindent=True, keyword_case="upper", indent_columns=True, indent_width=2
    )
)
print("\n" + "=" * 80 + "\n")

# Execute with DuckDB via GIQL engine
cursor = conn.execute(sql)
result = cursor.df()  # Get result as pandas DataFrame
print(f"Result: {len(result)} overlapping intervals from A")
result.head(10)

Transpiled SQL:
SELECT DISTINCT a.*
FROM
  features_a AS a,
  features_b AS b
WHERE (a."chrom" = b."chrom"
       AND a."start" < b."end"
       AND a."end" > b."start")


Result: 42616 overlapping intervals from A


,chrom,start,end,name,score,strand,signal_value,p_value,q_value,peak
0,chr22,37933998,37934242,.,1000,.,283.80615,-1.0,5.08726,123
1,chr20,267021,267251,.,1000,.,258.60836,-1.0,5.08726,126
2,chr20,63537989,63538200,.,1000,.,272.24062,-1.0,5.08726,98
3,chr14,25042978,25043164,.,1000,.,259.47262,-1.0,5.08726,106
4,chr15,64728285,64728479,.,1000,.,234.42653,-1.0,5.08726,96
5,chr20,34129746,34129928,.,1000,.,243.36217,-1.0,5.08726,101
6,chr20,48657805,48658001,.,1000,.,257.44302,-1.0,5.08726,93
7,chr14,38721933,38722118,.,1000,.,239.09691,-1.0,5.08726,98
8,chr14,23632038,23632231,.,1000,.,255.61314,-1.0,5.08726,91
9,chr16,11532213,11532470,.,1000,.,339.88370,-1.0,5.08726,124


### High overlap depth regions

Find regions with high overlap depth (where many intervals overlap) using a self-join.

In [4]:
giql_query = """
    SELECT DISTINCT a.*
    FROM features_a a, features_b b
    WHERE a.interval INTERSECTS b.interval
"""

sql = transpile(giql_query, tables=["features_a", "features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} overlapping intervals from A")
result.head(10)

Transpiled SQL:
SELECT DISTINCT
  a.*
FROM features_a AS a, features_b AS b
WHERE
  (
    a."chrom" = b."chrom" AND a."start" < b."end" AND a."end" > b."start"
  )


Result: 42616 overlapping intervals from A


,chrom,start,end,name,score,strand,signal_value,p_value,q_value,peak
0,chr19,45642140,45642400,.,1000,.,297.57428,-1.0,5.08726,128
1,chr11,518674,518971,.,1000,.,315.46469,-1.0,5.08726,149
2,chr20,50794798,50795030,.,1000,.,260.64664,-1.0,5.08726,115
3,chr19,46395603,46395857,.,1000,.,335.49048,-1.0,5.08726,134
4,chr9,34646905,34647110,.,1000,.,235.66436,-1.0,5.08726,100
5,chr14,53929955,53930179,.,1000,.,340.46918,-1.0,5.08726,115
6,chr12,6323309,6323518,.,1000,.,262.07653,-1.0,5.08726,99
7,chr14,32073682,32073873,.,1000,.,278.18167,-1.0,5.08726,89
8,chr20,59087928,59088111,.,1000,.,234.98679,-1.0,5.08726,92
9,chr15,79596083,79596368,.,1000,.,347.14815,-1.0,5.08726,144


---

## 2. WITHIN operator

Find features in A that are **completely contained within** features in B.

**GIQL Query**: `SELECT a.* FROM features_a a, features_b b WHERE a.interval WITHIN b.interval`

In [5]:
giql_query = """
    WITH overlap_counts AS (
        SELECT
            a.chrom,
            a.start,
            a."end",
            a.signal_value,
            COUNT(*) as depth
        FROM features_a a, features_b b
        WHERE a.interval INTERSECTS b.interval
        GROUP BY a.chrom, a.start, a."end", a.signal_value
    )
    SELECT *
    FROM overlap_counts
    WHERE depth > 2
    ORDER BY depth DESC
"""

sql = transpile(giql_query, tables=["features_a", "features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} intervals with overlap depth > 2")
print(f"Max overlap depth: {result['depth'].max() if len(result) > 0 else 0}")
result.head(10)

Transpiled SQL:
WITH overlap_counts AS (
  SELECT
    a.chrom,
    a.start,
    a."end",
    a.signal_value,
    COUNT(*) AS depth
  FROM features_a AS a, features_b AS b
  WHERE
    (
      a."chrom" = b."chrom" AND a."start" < b."end" AND a."end" > b."start"
    )
  GROUP BY
    a.chrom,
    a.start,
    a."end",
    a.signal_value
)
SELECT
  *
FROM overlap_counts
WHERE
  depth > 2
ORDER BY
  depth DESC


Result: 2 intervals with overlap depth > 2
Max overlap depth: 3


,chrom,start,end,signal_value,depth
0,chr14,105999349,105999944,240.51649,3
1,chr14,105999349,105999944,108.06722,3


---

## 3. CONTAINS operator

Find features in A that **completely contain** features in B.

**GIQL Query**: `SELECT a.* FROM features_a a, features_b b WHERE a.interval CONTAINS b.interval`

In [6]:
giql_query = """
    SELECT DISTINCT a.*, b.*
    FROM features_a a, features_b b
    WHERE a.interval WITHIN b.interval
"""

sql = transpile(giql_query, tables=["features_a", "features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} intervals from A contained within B")
result.head(10)

Transpiled SQL:
SELECT DISTINCT
  a.*,
  b.*
FROM features_a AS a, features_b AS b
WHERE
  (
    a."chrom" = b."chrom" AND a."start" >= b."start" AND a."end" <= b."end"
  )


Result: 30809 intervals from A contained within B


,chrom,start,end,name,score,strand,signal_value,p_value,q_value,peak,chrom_1,start_1,end_1,name_1,score_1,strand_1,signal_value_1,p_value_1,q_value_1,peak_1
0,chr19,43774483,43774712,.,1000,.,252.61723,-1.0,5.08726,103,chr19,43774432,43774742,.,951,.,28.41530,-1.0,5.10518,155
1,chr17,75615499,75615705,.,1000,.,244.31776,-1.0,5.08726,96,chr17,75615479,75615789,.,1000,.,26.62768,-1.0,5.10518,155
2,chr11,397249,397474,.,1000,.,237.92944,-1.0,5.08726,114,chr11,397230,397540,.,1000,.,31.74873,-1.0,5.10518,155
3,chr20,44116742,44116950,.,1000,.,292.51551,-1.0,5.08726,106,chr20,44116709,44116975,.,1000,.,201.73737,-1.0,5.10518,132
4,chr16,23434388,23434589,.,1000,.,267.80248,-1.0,5.08726,106,chr16,23434384,23434621,.,1000,.,148.78159,-1.0,5.10518,119
5,chr17,35804783,35805037,.,1000,.,314.27261,-1.0,5.08726,134,chr17,35804768,35805048,.,1000,.,113.42031,-1.0,5.10518,149
6,chr4,147866849,147867113,.,1000,.,395.15579,-1.0,5.08726,133,chr4,147866840,147867128,.,1000,.,191.86545,-1.0,5.10518,141
7,chr10,97759515,97759702,.,1000,.,236.46727,-1.0,5.08726,97,chr10,97759495,97759724,.,1000,.,140.81146,-1.0,5.10518,104
8,chr12,53701870,53702056,.,1000,.,253.82117,-1.0,5.08726,96,chr12,53701848,53702069,.,1000,.,131.82871,-1.0,5.10518,97
9,chr12,115398399,115398644,.,1000,.,296.60121,-1.0,5.08726,110,chr12,115398396,115398671,.,1000,.,135.44415,-1.0,5.10518,138


---

## 4. MERGE operator

**Combine overlapping intervals** from features_a into merged regions.

Similar to `bedtools merge`, this collapses overlapping genomic intervals.

**GIQL Query**: `SELECT MERGE(interval) FROM features_a`

In [7]:
giql_query = """
    SELECT DISTINCT a.*
    FROM features_a a, features_b b
    WHERE a.interval CONTAINS b.interval
"""

sql = transpile(giql_query, tables=["features_a", "features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} intervals from A that contain B")
result.head(10)

Transpiled SQL:
SELECT DISTINCT
  a.*
FROM features_a AS a, features_b AS b
WHERE
  (
    a."chrom" = b."chrom" AND a."start" <= b."start" AND a."end" >= b."end"
  )


Result: 4917 intervals from A that contain B


,chrom,start,end,name,score,strand,signal_value,p_value,q_value,peak
0,chr22,22559162,22559461,.,1000,.,378.39470,-1.0,5.08726,143
1,chr19,45642140,45642400,.,1000,.,297.57428,-1.0,5.08726,128
2,chr19,46395603,46395857,.,1000,.,335.49048,-1.0,5.08726,134
3,chr12,6323309,6323518,.,1000,.,262.07653,-1.0,5.08726,99
4,chr12,546987,547224,.,1000,.,273.49144,-1.0,5.08726,126
5,chr6,34154821,34155022,.,1000,.,258.93713,-1.0,5.08726,95
6,chr5,181184125,181184440,.,1000,.,277.71855,-1.0,5.08726,138
7,chr16,11532213,11532470,.,1000,.,339.88370,-1.0,5.08726,124
8,chr5,95781099,95781372,.,1000,.,255.82894,-1.0,5.08726,132
9,chr12,111396988,111397250,.,1000,.,406.49219,-1.0,5.08726,130


### MERGE with aggregation CTE

Compute statistics on merged intervals and filter for intervals that merged multiple features.

In [8]:
giql_query = """
    SELECT MERGE(interval)
    FROM features_b
"""

sql = transpile(giql_query, tables=["features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

n_original = conn.execute("SELECT COUNT(*) FROM features_b").fetchone()[0]
result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} merged intervals (from {n_original} original)")
result.head(10)

Transpiled SQL:
SELECT
  "chrom",
  MIN("start") AS start,
  MAX("end") AS end
FROM (
  SELECT
    *,
    SUM(is_new_cluster) OVER (PARTITION BY "chrom" ORDER BY "start" NULLS LAST) AS __giql_cluster_id
  FROM (
    SELECT
      *,
      CASE
        WHEN LAG("end") OVER (PARTITION BY "chrom" ORDER BY "start" NULLS LAST) >= "start"
        THEN 0
        ELSE 1
      END AS is_new_cluster
    FROM features_b
  ) AS lag_calc
) AS clustered
GROUP BY
  chrom,
  __giql_cluster_id
ORDER BY
  "chrom" NULLS LAST,
  "start" NULLS LAST


Result: 45630 merged intervals (from 46196 original)


,chrom,start,end
0,chr1,186654,186964
1,chr1,267979,268128
2,chr1,850424,850734
3,chr1,869711,870021
4,chr1,912864,913174
5,chr1,919635,919945
6,chr1,931626,931936
7,chr1,938195,938352
8,chr1,951414,951724
9,chr1,976080,976390


### MERGE with distance parameter

Merge intervals within 1000bp of each other.

In [9]:
giql_query = """
    WITH merged_intervals AS (
        SELECT
            MERGE(interval),
            COUNT(*) as interval_count,
            AVG(signal_value) as avg_signal
        FROM features_b
    )
    SELECT *
    FROM merged_intervals
    WHERE interval_count > 1
"""

sql = transpile(giql_query, tables=["features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} merged intervals with more than 1 original interval")
result.head(10)

Transpiled SQL:
WITH merged_intervals AS (
  SELECT
    "chrom",
    MIN("start") AS start,
    MAX("end") AS end,
    COUNT(*) AS interval_count,
    AVG(signal_value) AS avg_signal
  FROM (
    SELECT
      *,
      SUM(is_new_cluster) OVER (PARTITION BY "chrom" ORDER BY "start" NULLS LAST) AS __giql_cluster_id
    FROM (
      SELECT
        *,
        CASE
          WHEN LAG("end") OVER (PARTITION BY "chrom" ORDER BY "start" NULLS LAST) >= "start"
          THEN 0
          ELSE 1
        END AS is_new_cluster
      FROM features_b
    ) AS lag_calc
  ) AS clustered
  GROUP BY
    chrom,
    __giql_cluster_id
  ORDER BY
    "chrom" NULLS LAST,
    "start" NULLS LAST
)
SELECT
  *
FROM merged_intervals
WHERE
  interval_count > 1


Result: 559 merged intervals with more than 1 original interval


,chrom,start,end,interval_count,avg_signal
0,chr1,1300073,1300592,2,14.844510
1,chr1,8374912,8375330,2,28.608395
2,chr1,9717307,9717819,2,13.833490
3,chr1,14782921,14783438,2,7.107920
4,chr1,18362141,18362677,2,9.312525
5,chr1,19445773,19446283,2,16.590025
6,chr1,23958874,23959385,2,10.616965
7,chr1,24157968,24158482,2,11.985625
8,chr1,27392817,27393359,2,11.789830
9,chr1,30106143,30106664,2,31.694800


---

## 5. CLUSTER operator

**Assign cluster IDs** to overlapping intervals from features_a.

Similar to `bedtools cluster`, this groups overlapping genomic intervals.

**GIQL Query**: `SELECT *, CLUSTER(interval) AS cluster_id FROM features_a`

In [10]:
giql_query = """
    SELECT MERGE(interval, 10000)
    FROM features_b
"""

sql = transpile(giql_query, tables=["features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

n_original = conn.execute("SELECT COUNT(*) FROM features_b").fetchone()[0]
result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} merged intervals (10000bp distance)")
print(f"Compared to: {n_original} original intervals")
result.head(10)

Transpiled SQL:
SELECT
  "chrom",
  MIN("start") AS start,
  MAX("end") AS end
FROM (
  SELECT
    *,
    SUM(is_new_cluster) OVER (PARTITION BY "chrom" ORDER BY "start" NULLS LAST) AS __giql_cluster_id
  FROM (
    SELECT
      *,
      CASE
        WHEN LAG("end") OVER (PARTITION BY "chrom" ORDER BY "start" NULLS LAST) + 10000 >= "start"
        THEN 0
        ELSE 1
      END AS is_new_cluster
    FROM features_b
  ) AS lag_calc
) AS clustered
GROUP BY
  chrom,
  __giql_cluster_id
ORDER BY
  "chrom" NULLS LAST,
  "start" NULLS LAST


Result: 34097 merged intervals (10000bp distance)
Compared to: 46196 original intervals


,chrom,start,end
0,chr1,186654,186964
1,chr1,267979,268128
2,chr1,850424,850734
3,chr1,869711,870021
4,chr1,912864,919945
5,chr1,931626,938352
6,chr1,951414,951724
7,chr1,976080,984477
8,chr1,1013987,1015656
9,chr1,1063787,1064097


### CLUSTER with distance parameter

Cluster intervals within 1000bp of each other.

In [11]:
giql_query = """
    SELECT
        chrom,
        start,
        "end",
        signal_value,
        CLUSTER(interval) AS cluster_id
    FROM features_a
    ORDER BY chrom, start
"""

sql = transpile(giql_query, tables=["features_a"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} intervals with cluster assignments")
print(f"Number of unique clusters: {result['cluster_id'].nunique()}")
result.head(10)

Transpiled SQL:
SELECT
  chrom,
  start,
  "end",
  signal_value,
  SUM(is_new_cluster) OVER (PARTITION BY "chrom" ORDER BY "start" NULLS LAST) AS cluster_id
FROM (
  SELECT
    chrom,
    start,
    "end",
    signal_value,
    CASE
      WHEN LAG("end") OVER (PARTITION BY "chrom" ORDER BY "start" NULLS LAST) >= "start"
      THEN 0
      ELSE 1
    END AS is_new_cluster
  FROM features_a
) AS lag_calc
ORDER BY
  chrom,
  start


Result: 60893 intervals with cluster assignments
Number of unique clusters: 5495


,chrom,start,end,signal_value,cluster_id
0,chr1,181368,181564,27.54796,1.0
1,chr1,186650,186846,19.93098,2.0
2,chr1,267909,268105,88.00863,3.0
3,chr1,586106,586302,35.02027,4.0
4,chr1,729261,729457,23.29415,5.0
5,chr1,778812,779008,56.97663,6.0
6,chr1,850473,850669,27.08243,7.0
7,chr1,858056,858252,15.55869,8.0
8,chr1,869860,869991,168.87294,9.0
9,chr1,904689,904883,167.56897,10.0


---

## 6. DISTANCE operator

**Calculate genomic distances** between intervals from `features_a` and `features_b`.

Similar to `bedtools closest -d`, this calculates the distance in base pairs between genomic intervals.

**GIQL Query**: `SELECT a.*, b.*, DISTANCE(a.interval, b.interval) AS distance FROM features_a a, features_b b`

In [12]:
giql_query = """
    SELECT
        chrom,
        start,
        "end",
        signal_value,
        CLUSTER(interval, 50000) AS cluster_id
    FROM features_a
    ORDER BY chrom, start
"""

sql = transpile(giql_query, tables=["features_a"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} intervals with cluster assignments (50000bp distance)")
print(f"Number of unique clusters: {result['cluster_id'].nunique()}")
result.head(10)

Transpiled SQL:
SELECT
  chrom,
  start,
  "end",
  signal_value,
  SUM(is_new_cluster) OVER (PARTITION BY "chrom" ORDER BY "start" NULLS LAST) AS cluster_id
FROM (
  SELECT
    chrom,
    start,
    "end",
    signal_value,
    CASE
      WHEN LAG("end") OVER (PARTITION BY "chrom" ORDER BY "start" NULLS LAST) + 50000 >= "start"
      THEN 0
      ELSE 1
    END AS is_new_cluster
  FROM features_a
) AS lag_calc
ORDER BY
  chrom,
  start


Result: 60893 intervals with cluster assignments (50000bp distance)
Number of unique clusters: 1376


,chrom,start,end,signal_value,cluster_id
0,chr1,181368,181564,27.54796,1.0
1,chr1,186650,186846,19.93098,1.0
2,chr1,267909,268105,88.00863,2.0
3,chr1,586106,586302,35.02027,3.0
4,chr1,729261,729457,23.29415,4.0
5,chr1,778812,779008,56.97663,4.0
6,chr1,850473,850669,27.08243,5.0
7,chr1,858056,858252,15.55869,5.0
8,chr1,869860,869991,168.87294,5.0
9,chr1,904689,904883,167.56897,5.0


### Signed Distance (Directional)

Calculate **directional distances** where negative values indicate upstream features and positive values indicate downstream features, similar to `bedtools closest -D ref`.

In [13]:
giql_query = """
    WITH distances AS (
        SELECT
            a.chrom,
            a.start AS a_start,
            a."end" AS a_end,
            b.start AS b_start,
            b."end" AS b_end,
            DISTANCE(a.interval, b.interval, signed=true) AS signed_distance,
            ROW_NUMBER() OVER (
                PARTITION BY a.chrom, a.start, a."end"
                ORDER BY ABS(DISTANCE(a.interval, b.interval, signed=true))
            ) AS rank
        FROM features_a a
        CROSS JOIN features_b b
        WHERE a.chrom = b.chrom
            AND a.chrom = 'chr1'
            AND a.start < 500000
    )
    SELECT
        chrom,
        a_start,
        a_end,
        b_start,
        b_end,
        signed_distance,
        CASE
            WHEN signed_distance < 0 THEN 'upstream'
            WHEN signed_distance > 0 THEN 'downstream'
            ELSE 'overlap'
        END AS direction
    FROM distances
    WHERE rank = 1
    ORDER BY chrom, a_start
"""

sql = transpile(giql_query, tables=["features_a", "features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} features with directional distances")
print(f"Upstream features: {(result['signed_distance'] < 0).sum()}")
print(f"Downstream features: {(result['signed_distance'] > 0).sum()}")
print(f"Overlapping features: {(result['signed_distance'] == 0).sum()}")
result.head(10)

Transpiled SQL:
WITH distances AS (
  SELECT
    a.chrom,
    a.start AS a_start,
    a."end" AS a_end,
    b.start AS b_start,
    b."end" AS b_end,
    CASE
      WHEN a."chrom" <> b."chrom"
      THEN NULL
      WHEN a."start" < b."end" AND a."end" > b."start"
      THEN 0
      WHEN a."end" <= b."start"
      THEN (
        b."start" - a."end"
      )
      ELSE -(
        a."start" - b."end"
      )
    END AS signed_distance,
    ROW_NUMBER() OVER (
      PARTITION BY a.chrom, a.start, a."end"
      ORDER BY ABS(
        CASE
          WHEN a."chrom" <> b."chrom"
          THEN NULL
          WHEN a."start" < b."end" AND a."end" > b."start"
          THEN 0
          WHEN a."end" <= b."start"
          THEN (
            b."start" - a."end"
          )
          ELSE -(
            a."start" - b."end"
          )
        END
      )
    ) AS rank
  FROM features_a AS a
  CROSS JOIN features_b AS b
  WHERE
    a.chrom = b.chrom AND a.chrom = 'chr1' AND a.start < 500000
)
SELECT
  

,chrom,a_start,a_end,b_start,b_end,signed_distance,direction
0,chr1,181368,181564,186654,186964,5090,downstream
1,chr1,186650,186846,186654,186964,0,overlap
2,chr1,267909,268105,267979,268128,0,overlap


---

## 7. NEAREST operator

**Find the k-nearest genomic features** using the NEAREST operator.

Similar to `bedtools closest`, this finds the closest features to query intervals. The NEAREST operator eliminates the need to write complex window functions for k-nearest neighbor queries.

**Key Features**:
- Find k-nearest features (k=1 for closest, k>1 for multiple neighbors)
- Directional queries with `signed=true` (upstream vs downstream)
- Distance-constrained queries with `max_distance`
- Strand-specific queries with `stranded=true`
- Implicit reference resolution in LATERAL joins (automatically uses outer table's interval)

**Database Support**:
- **DuckDB**: Full support (LATERAL joins)
- **SQLite**: Standalone mode only (no LATERAL support)

**GIQL Query**: `SELECT * FROM peaks CROSS JOIN LATERAL NEAREST(genes, k=3)`

In [14]:
giql_query = """
    SELECT
        a.chrom,
        a.start AS a_start,
        a."end" AS a_end,
        b.start AS b_start,
        b."end" AS b_end,
        DISTANCE(a.interval, b.interval) AS distance
    FROM features_a a, features_b b
    WHERE a.chrom = b.chrom
        AND a.chrom = 'chr1'
        AND a.start BETWEEN 180000 AND 190000
        AND b.start BETWEEN 180000 AND 200000
    ORDER BY a.start, b.start
"""

sql = transpile(giql_query, tables=["features_a", "features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} distance calculations")
print("\nExample distances (0 = overlap, positive = gap in base pairs):")
result.head(10)

Transpiled SQL:
SELECT
  a.chrom,
  a.start AS a_start,
  a."end" AS a_end,
  b.start AS b_start,
  b."end" AS b_end,
  CASE
    WHEN a."chrom" <> b."chrom"
    THEN NULL
    WHEN a."start" < b."end" AND a."end" > b."start"
    THEN 0
    WHEN a."end" <= b."start"
    THEN (
      b."start" - a."end"
    )
    ELSE (
      a."start" - b."end"
    )
  END AS distance
FROM features_a AS a, features_b AS b
WHERE
  a.chrom = b.chrom
  AND a.chrom = 'chr1'
  AND a.start BETWEEN 180000 AND 190000
  AND b.start BETWEEN 180000 AND 200000
ORDER BY
  a.start,
  b.start


Result: 2 distance calculations

Example distances (0 = overlap, positive = gap in base pairs):


,chrom,a_start,a_end,b_start,b_end,distance
0,chr1,181368,181564,186654,186964,5090
1,chr1,186650,186846,186654,186964,0


### Finding k-Nearest Features

Find the **3 closest features** in B for each feature in A using `k=3`.

In [15]:
giql_query = """
    SELECT
        a.chrom,
        a.start AS a_start,
        a."end" AS a_end,
        a.signal_value AS a_signal,
        b.start AS b_start,
        b."end" AS b_end,
        b.signal_value AS b_signal,
        distance
    FROM features_a a, NEAREST(features_b, k=1) b
    WHERE a.chrom = 'chr1'
        AND a.start < 1000000
    ORDER BY a.chrom, a.start
"""

sql = transpile(giql_query, tables=["features_a", "features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()
print(f"Result: {len(result)} features from A with their closest feature in B")
print(f"Average distance to closest feature: {result['distance'].mean():.1f} bp")
print(f"Features overlapping with closest: {(result['distance'] == 0).sum()}")
result.head(10)

Transpiled SQL:
SELECT
  a.chrom,
  a.start AS a_start,
  a."end" AS a_end,
  a.signal_value AS a_signal,
  b.start AS b_start,
  b."end" AS b_end,
  b.signal_value AS b_signal,
  distance
FROM features_a AS a, (
  SELECT
    features_b.*,
    CASE
      WHEN a."chrom" <> features_b."chrom"
      THEN NULL
      WHEN a."start" < features_b."end" AND a."end" > features_b."start"
      THEN 0
      WHEN a."end" <= features_b."start"
      THEN (
        features_b."start" - a."end"
      )
      ELSE (
        a."start" - features_b."end"
      )
    END AS distance
  FROM features_b
  WHERE
    a."chrom" = features_b."chrom"
  ORDER BY
    ABS(
      CASE
        WHEN a."chrom" <> features_b."chrom"
        THEN NULL
        WHEN a."start" < features_b."end" AND a."end" > features_b."start"
        THEN 0
        WHEN a."end" <= features_b."start"
        THEN (
          features_b."start" - a."end"
        )
        ELSE (
          a."start" - features_b."end"
        )
      END
   

,chrom,a_start,a_end,a_signal,b_start,b_end,b_signal,distance
0,chr1,181368,181564,27.54796,186654,186964,14.61908,5090
1,chr1,186650,186846,19.93098,186654,186964,14.61908,0
2,chr1,267909,268105,88.00863,267979,268128,49.39366,0
3,chr1,586106,586302,35.02027,850424,850734,15.07029,264122
4,chr1,729261,729457,23.29415,850424,850734,15.07029,120967
5,chr1,778812,779008,56.97663,850424,850734,15.07029,71416
6,chr1,850473,850669,27.08243,850424,850734,15.07029,0
7,chr1,858056,858252,15.55869,850424,850734,15.07029,7322
8,chr1,869860,869991,168.87294,869711,870021,12.42156,0
9,chr1,904689,904883,167.56897,912864,913174,31.07865,7981


### Directional Queries with signed=true

Find the **3 nearest upstream or downstream features** using `signed=true`.

Negative distances indicate upstream features (B is before A), positive distances indicate downstream features (B is after A).

In [16]:
giql_query = """
    SELECT
        a.chrom,
        a.start AS a_start,
        a."end" AS a_end,
        a.signal_value AS a_signal,
        b.start AS b_start,
        b."end" AS b_end,
        b.signal_value AS b_signal,
        distance
    FROM features_a a, NEAREST(features_b, k=3) b
    WHERE a.chrom = 'chr1'
        AND a.start < 500000
    ORDER BY a.chrom, a.start, distance
"""

sql = transpile(giql_query, tables=["features_a", "features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()

grouped = result.groupby(["a_start", "a_end"]).size()
print(f"Result: {len(result)} total rows (up to 3 neighbors per feature in A)")
print(f"Number of features from A: {len(grouped)}")
print(f"Average neighbors per feature: {grouped.mean():.1f}")
result.head(15)

Transpiled SQL:
SELECT
  a.chrom,
  a.start AS a_start,
  a."end" AS a_end,
  a.signal_value AS a_signal,
  b.start AS b_start,
  b."end" AS b_end,
  b.signal_value AS b_signal,
  distance
FROM features_a AS a, (
  SELECT
    features_b.*,
    CASE
      WHEN a."chrom" <> features_b."chrom"
      THEN NULL
      WHEN a."start" < features_b."end" AND a."end" > features_b."start"
      THEN 0
      WHEN a."end" <= features_b."start"
      THEN (
        features_b."start" - a."end"
      )
      ELSE (
        a."start" - features_b."end"
      )
    END AS distance
  FROM features_b
  WHERE
    a."chrom" = features_b."chrom"
  ORDER BY
    ABS(
      CASE
        WHEN a."chrom" <> features_b."chrom"
        THEN NULL
        WHEN a."start" < features_b."end" AND a."end" > features_b."start"
        THEN 0
        WHEN a."end" <= features_b."start"
        THEN (
          features_b."start" - a."end"
        )
        ELSE (
          a."start" - features_b."end"
        )
      END
   

,chrom,a_start,a_end,a_signal,b_start,b_end,b_signal,distance
0,chr1,181368,181564,27.54796,186654,186964,14.61908,5090
1,chr1,181368,181564,27.54796,267979,268128,49.39366,86415
2,chr1,181368,181564,27.54796,850424,850734,15.07029,668860
3,chr1,186650,186846,19.93098,186654,186964,14.61908,0
4,chr1,186650,186846,19.93098,267979,268128,49.39366,81133
5,chr1,186650,186846,19.93098,850424,850734,15.07029,663578
6,chr1,267909,268105,88.00863,267979,268128,49.39366,0
7,chr1,267909,268105,88.00863,186654,186964,14.61908,80945
8,chr1,267909,268105,88.00863,850424,850734,15.07029,582319


### Distance-Constrained Queries with max_distance

Find up to **5 nearest features within 50kb** using `max_distance=50000`.

This is useful for finding nearby regulatory elements or associated genes within a biologically relevant distance.

In [17]:
giql_query = """
    SELECT
        a.chrom,
        a.start AS a_start,
        a."end" AS a_end,
        b.start AS b_start,
        b."end" AS b_end,
        distance,
        CASE
            WHEN distance < 0 THEN 'upstream'
            WHEN distance > 0 THEN 'downstream'
            ELSE 'overlap'
        END AS direction
    FROM features_a a, NEAREST(features_b, k=3, signed=true) b
    WHERE a.chrom = 'chr1'
        AND a.start < 500000
        AND direction = 'upstream'
    ORDER BY a.chrom, a.start, ABS(distance)
"""

sql = transpile(giql_query, tables=["features_a", "features_b"])
print("Transpiled SQL:")
print(sqlglot.transpile(sql, pretty=True)[0])
print("\n" + "=" * 80 + "\n")

result = conn.execute(sql).fetchdf()

upstream = (result["distance"] < 0).sum()
downstream = (result["distance"] > 0).sum()
overlap = (result["distance"] == 0).sum()

print(f"Result: {len(result)} total nearest features")
print(f"Upstream features (distance < 0): {upstream}")
print(f"Downstream features (distance > 0): {downstream}")
print(f"Overlapping features (distance = 0): {overlap}")
result.head(15)

Transpiled SQL:
SELECT
  a.chrom,
  a.start AS a_start,
  a."end" AS a_end,
  b.start AS b_start,
  b."end" AS b_end,
  distance,
  CASE
    WHEN distance < 0
    THEN 'upstream'
    WHEN distance > 0
    THEN 'downstream'
    ELSE 'overlap'
  END AS direction
FROM features_a AS a, (
  SELECT
    features_b.*,
    CASE
      WHEN a."chrom" <> features_b."chrom"
      THEN NULL
      WHEN a."start" < features_b."end" AND a."end" > features_b."start"
      THEN 0
      WHEN a."end" <= features_b."start"
      THEN (
        features_b."start" - a."end"
      )
      ELSE -(
        a."start" - features_b."end"
      )
    END AS distance
  FROM features_b
  WHERE
    a."chrom" = features_b."chrom"
  ORDER BY
    ABS(
      CASE
        WHEN a."chrom" <> features_b."chrom"
        THEN NULL
        WHEN a."start" < features_b."end" AND a."end" > features_b."start"
        THEN 0
        WHEN a."end" <= features_b."start"
        THEN (
          features_b."start" - a."end"
        )
    

,chrom,a_start,a_end,b_start,b_end,distance,direction
0,chr1,267909,268105,186654,186964,-80945,upstream


---

## Summary

This demo showcased all 7 GIQL operators:

1. **INTERSECTS**: Binary predicate for overlapping intervals
2. **WITHIN**: Binary predicate for containment (A within B)
3. **CONTAINS**: Binary predicate for containment (A contains B)
4. **MERGE**: Aggregation operator to combine overlapping intervals
5. **CLUSTER**: Aggregation operator to assign cluster IDs to overlapping intervals
6. **DISTANCE**: UDF operator to calculate genomic distances between intervals
7. **NEAREST**: Table-valued function for finding k-nearest genomic features

Each operator was:
- Written in GIQL syntax
- Transpiled to standard SQL
- Executed using DuckDB

This demonstrates how GIQL provides a high-level, intuitive syntax for genomic interval operations while maintaining compatibility with standard SQL engines through transpilation.